# 1. 환경 세팅

In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [7]:
import boto3
import os
import json
from botocore.config import Config
import botocore 
from pprint import pprint
from termcolor import colored

session = boto3.Session()

region_name='us-west-2'

retry_config = Config(
    # region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    region_name=region_name,
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

# modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = "application/json"
contentType = "application/json"

bedrock = boto3.client(service_name='bedrock', region_name=region_name)
boto3_bedrock = boto3.client(service_name='bedrock-runtime',config=retry_config)

model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'),
 ('Claude Instant', 'anthropic.claude-instant-v1'),
 ('Claude', 'anthropic.claude-v2:1'),
 ('Claude', 'anthropic.claude-v2'),
 ('Claude 3 Sonnet', 'anthropic.clau

---

## `InvokeModel` body and output

The `invoke_model()` method of the Amazon Bedrock runtime client (`InvokeModel` API) will be the primary method we use for most of our Text Generation and Processing tasks - whichever model we're using.

Although the method is shared, the format of input and output varies depending on the foundation model used - as described below:

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Amazon Titan Large

#### Input
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### Output

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### AI21 Jurassic (Grande and Jumbo) 

#### Input

```json
{
    "prompt": "<prompt>",
    "maxTokens": 200,
    "temperature": 0.5,
    "topP": 0.5,
    "stopSequences": [],
    "countPenalty": {"scale": 0},
    "presencePenalty": {"scale": 0},
    "frequencyPenalty": {"scale": 0}
}
```

#### Output

```json
{
    "id": 1234,
    "prompt": {
        "text": "<prompt>",
        "tokens": [
            {
                "generatedToken": {
                    "token": "\u2581who\u2581is",
                    "logprob": -12.980147361755371,
                    "raw_logprob": -12.980147361755371
                },
                "topTokens": null,
                "textRange": {"start": 0, "end": 6}
            },
            //...
        ]
    },
    "completions": [
        {
            "data": {
                "text": "<output>",
                "tokens": [
                    {
                        "generatedToken": {
                            "token": "<|newline|>",
                            "logprob": 0.0,
                            "raw_logprob": -0.01293118204921484
                        },
                        "topTokens": null,
                        "textRange": {"start": 0, "end": 1}
                    },
                    //...
                ]
            },
            "finishReason": {"reason": "endoftext"}
        }
    ]
}
```

### Stability AI Stable Diffusion XL

#### Input

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### Output

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```

# 2. Titan Embedding 및 LLM 인 Claude-v3 모델 로딩

## LLM 로딩 (Claude-v3)

In [19]:
def image_to_base64(img):
    import base64
    """Converts a PIL Image or local image file path to a base64 string"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")


def bedrock_streamer(response):
    stream = response.get('body')
    answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1
    return answer

In [20]:
from botocore.config import Config

session = boto3.Session()

retry_config = Config(
    region_name=region_name,
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)


bedrock_runtime = session.client(
    service_name="bedrock-runtime",
    config=retry_config
)

In [21]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """너는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
image_path = None

In [22]:
if image_path is None:
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_data},
                ],
            }
        ],
    }
else:
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_to_base64(image_path),
                        },
                    },
                    {"type": "text", "text": prompt_data},
                ],
            }
        ],
    }


body = json.dumps(prompt_config)

In [24]:
from IPython.display import clear_output, display, display_markdown, Markdown

modelId = "anthropic.claude-3-haiku-20240307-v1:0"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    results=bedrock_streamer(response)

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


생명보험과 손해보험의 주요 차이점은 다음과 같습니다:

1. 보장 범위:
- 생명보험은 사망, 질병, 상해 등 피보험자의 신체적 위험을 보장합니다.
- 손해보험은 재산상의 손실이나 배상책임 등 피보험자의 재산적 위험을 보장합니다.

2. 보험금 지급 시기:
- 생명보험은 피보험자의 사망, 질병, 상해 등이 발생했을 때 보험금을 지급합니다.
- 손해보험은 사고 발생 시 실제 손실 금액을 보상합니다.

3. 보험금 지급 방식:
- 생명보험은 일정 금액의 보험금을 지급합니다.
- 손해보험은 실제 손실 금액을 보상합니다.

4. 보험 목적:
- 생명보험은 피보험자의 생명과 건강을 보장하는 것이 주목적입니다.
- 손해보험은 재산상의 손실을 보상하는 것이 주목적입니다.

이처럼 생명보험과 손해보험은 보장 범위, 보험금 지급 시기와 방식, 보험 목적 등에서 차이가 있습니다. 고객의 필요에 따라 적절한 보험 상품을 선택하는 것이 중요합니다.

In [28]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [30]:
llm_text = BedrockChat(
    model_id=modelId,
    streaming=True,
    region_name=region_name,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999
    }
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7fa7a79d0580>, region_name='us-west-2', model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'anthropic_version': 'bedrock-2023-05-31', 'max_tokens': 4096, 'temperature': 0, 'top_k': 350, 'top_p': 0.999}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fa7b2f78670>])

In [32]:
# If you'd like to try your own prompt, edit this parameter!
prompt_template = """

Human: 너는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

<context>
{history}
</context>

Question: {input} using within <context></context>


Assistant:
"""

In [33]:
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['history', 'input']
)

In [34]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm_text, 
    verbose=True, 
    memory=ConversationBufferMemory(),
    prompt=PROMPT
)

conversation.predict(input=prompt_data)



> Entering new ConversationChain chain...
Prompt after formatting:


Human: 너는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

<context>

</context>

Question: 너는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요. using within <context></context>


Assistant:

<context>
생명보험과 손해보험은 보장하는 위험의 종류와 보험금 지급 방식에서 차이가 있습니다. 생명보험은 사망, 질병, 상해 등 개인의 생명과 관련된 위험을 보장하며, 보험금은 사고 발생 시 일시금으로 지급됩니다. 반면 손해보험은 재산상의 손실이나 배상책임 등 재무적 손실을 보장하며, 실제 손실 금액을 기준으로 보험금이 지급됩니다.
</context>

생명보험과 손해보험의 주요 차이점은 다음과 같습니다:

1. 보장 대상: 생명보험은 개인의 생명과 건강을 보장하지만, 손해보험은 재산상의 손실이나 배상책임을 보장합니다.

2. 보험금 지급 방식: 생명보험은 사고 발생 시 일시금으로 보험금이 지급되지만, 손해보험은 실제 손실 금액을 기준으로 보험금이 지급됩니다.

3. 보험료 산정 방식: 생명보험은 연령, 성별, 건강상태 등을 고려하여 보험료가 결정되지만, 손해보험은 위험 요인, 과거 사고 경력 등을 기준으로 보험료가 산정됩니다.

4. 보험 기간: 생명보험은 장기 계약이 일반적이지만, 손해보험은 단기 계약이 많습니다.

이와 같은 차이점으로 인해 고객의 필요에 따라 생명보험과 손해보험을 선택할 수 있습니다. 보험 가입 시 이러한 차이점을 고려하여 적절한 보험 상품을 선택하는 것이 중요합니다.
> Finished chain.


'<context>\n생명보험과 손해보험은 보장하는 위험의 종류와 보험금 지급 방식에서 차이가 있습니다. 생명보험은 사망, 질병, 상해 등 개인의 생명과 관련된 위험을 보장하며, 보험금은 사고 발생 시 일시금으로 지급됩니다. 반면 손해보험은 재산상의 손실이나 배상책임 등 재무적 손실을 보장하며, 실제 손실 금액을 기준으로 보험금이 지급됩니다.\n</context>\n\n생명보험과 손해보험의 주요 차이점은 다음과 같습니다:\n\n1. 보장 대상: 생명보험은 개인의 생명과 건강을 보장하지만, 손해보험은 재산상의 손실이나 배상책임을 보장합니다.\n\n2. 보험금 지급 방식: 생명보험은 사고 발생 시 일시금으로 보험금이 지급되지만, 손해보험은 실제 손실 금액을 기준으로 보험금이 지급됩니다.\n\n3. 보험료 산정 방식: 생명보험은 연령, 성별, 건강상태 등을 고려하여 보험료가 결정되지만, 손해보험은 위험 요인, 과거 사고 경력 등을 기준으로 보험료가 산정됩니다.\n\n4. 보험 기간: 생명보험은 장기 계약이 일반적이지만, 손해보험은 단기 계약이 많습니다.\n\n이와 같은 차이점으로 인해 고객의 필요에 따라 생명보험과 손해보험을 선택할 수 있습니다. 보험 가입 시 이러한 차이점을 고려하여 적절한 보험 상품을 선택하는 것이 중요합니다.'

## Function call

In [35]:
from function_call import add_tools, run_loop

In [45]:
def create_prompt(tools_string, user_input):
    prompt_template = f"""
In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

Human:
{user_input}


Assistant:
"""
    return prompt_template

In [47]:
user_input = "Can you check the weather for me in Paris, France?"
tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt, region_name)


In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

            <tool_description>
            <tool_name>get_weather</tool_name>
            <description>
            Returns weather data for a given latitude and longitude. </description>
            <parameters>
            <parameter>
            <name>latitude</name>
            <type>string</type>
            <description>The latitude coordinate as a string</description>
            </parameter> <parameter>
            <name>longitude</name>
            <type>string</type>
            <description>The longitude coordinate as a string</description>
         

In [ ]:
user_input = "한국의 오늘 날씨는 어떻게 되나요?"
tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt, region_name)


In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

            <tool_description>
            <tool_name>get_weather</tool_name>
            <description>
            Returns weather data for a given latitude and longitude. </description>
            <parameters>
            <parameter>
            <name>latitude</name>
            <type>string</type>
            <description>The latitude coordinate as a string</description>
            </parameter> <parameter>
            <name>longitude</name>
            <type>string</type>
            <description>The longitude coordinate as a string</description>
         

In [19]:
!pip list|grep defusedxml

defusedxml                    0.7.1


In [20]:
!pip list|grep requests

requests                      2.31.0
requests-kerberos             0.14.0
